## Find and Load Data to SQL :  US and Global Energy Generation and Consumption 

In [1]:
## Dedependiecies 

# Web scrape
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import psycopg2

# SQL
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from flask_sqlalchemy import SQLAlchemy

# General
import pandas as pd
import numpy as np

## Web scraping

In [2]:
# Open browser with splinter 
# (Headless = True) Thus we do not see the browser oepning and closing
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# HTML Object and Beautiful Soup Parseing
url = 'https://www.nei.org/resources/statistics/state-electricity-generation-fuel-shares'
browser.visit(url)

In [4]:
# HTML Object and Beautiful Soup Parseing
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve and collect all desired data
webTable = soup.find_all('tr')

# When in doube print() it out
# All desired data is in one line, thus have to split
us_percent_list = []
temp_list = []
list_element = ""
tableData = []

for result in webTable:
    tableData = result.find_all('td')
    temp_list = []
    for data in tableData:
        list_element = data.get_text()
        temp_list.append(list_element)
    if not temp_list:
        pass
    else:
        us_percent_list.append(temp_list)

print(us_percent_list)

[['Alabama', '31.4', '19.2', '38.7', '0.0', '7.8', '0.0', '0.3', '0.0', '2.5'], ['Alaska', '0.0', '10.6', '49.9', '14.4', '22.0', '0.0', '0.0', '2.4', '0.6'], ['Arizona', '28.1', '20.4', '40.9', '0.1', '5.4', '0.0', '4.5', '0.5', '0.2'], ['Arkansas', '22.0', '37.9', '33.0', '0.1', '4.5', '0.0', '0.3', '0.0', '2.2'], ['California', '7.9', '0.1', '42.4', '0.0', '19.5', '5.6', '14.0', '7.3', '3.2'], ['Colorado', '0.0', '45.1', '30.4', '0.0', '2.5', '0.0', '2.1', '19.4', '0.4'], ['Connecticut', '41.7', '0.1', '52.8', '0.2', '1.3', '0.0', '0.4', '0.0', '3.3'], ['Delaware', '0.0', '2.3', '94.7', '0.8', '0.0', '0.0', '1.0', '0.1', '1.0'], ['District of Columbia', '0.0', '0.0', '27.1', '0.0', '0.0', '0.0', '15.1', '0.0', '57.8'], ['Florida', '11.8', '9.1', '74.3', '0.1', '0.1', '0.0', '1.6', '0.0', '3.1'], ['Georgia', '26.3', '19.9', '45.7', '0.1', '2.3', '0.0', '1.6', '0.0', '4.1'], ['Hawaii', '0.0', '13.2', '0.1', '69.3', '0.7', '1.1', '2.7', '6.0', '6.9'], ['Idaho', '0.0', '0.1', '21.1', '0

In [6]:
# Convert data into df and clean
#US_percentage_df = pd.DataFrame(us_percent_list, columns = tableHeaders)
US_percentage_df = pd.DataFrame(us_percent_list, columns = ["State", "Nuclear", "Coal", "Natural Gas", 
                                                            "Petroleum", "Hydro", "Geothermal", 
                                                            "Solar - PV", "Wind", "Biomass and Other"])


print(f'The length of this df is: {len(US_percentage_df)}')
US_percentage_df.head()

#### ValueError: 10 columns passed, passed data had 510 columns  ####

The length of this df is: 51


,State,Nuclear,Coal,Natural Gas,Petroleum,Hydro,Geothermal,Solar - PV,Wind,Biomass and Other
0,Alabama,31.4,19.2,38.7,0.0,7.8,0.0,0.3,0.0,2.5
1,Alaska,0.0,10.6,49.9,14.4,22.0,0.0,0.0,2.4,0.6
2,Arizona,28.1,20.4,40.9,0.1,5.4,0.0,4.5,0.5,0.2
3,Arkansas,22.0,37.9,33.0,0.1,4.5,0.0,0.3,0.0,2.2
4,California,7.9,0.1,42.4,0.0,19.5,5.6,14.0,7.3,3.2


In [7]:
# Finished scraping from the web, quit browser in background
browser.quit()

## CSV: Load  and Clean

### CSV Source 1

In [8]:
# read from local file
raw_csv_1 = "./Resources/US/annual_generation_state.csv"
CSV_df_1 = pd.read_csv(raw_csv_1, skiprows=4, sep = ',', encoding='ISO-8859-1')
print(f'Length of CSV1 is: {len(CSV_df_1)}')
# Display head of data set to be sure it was loaded correctly
CSV_df_1.head()

Length of CSV1 is: 53756


,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1990,AK,Total Electric Power Industry,Total,"5,599,506",NaN,NaN,NaN,NaN,NaN,NaN
1,1990,AK,Total Electric Power Industry,Coal,"510,573",NaN,NaN,NaN,NaN,NaN,NaN
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,"974,521",NaN,NaN,NaN,NaN,NaN,NaN
3,1990,AK,Total Electric Power Industry,Natural Gas,"3,466,261",NaN,NaN,NaN,NaN,NaN,NaN
4,1990,AK,Total Electric Power Industry,Petroleum,"497,116",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
annual_US_generation_df = CSV_df_1[['YEAR','STATE','TYPE OF PRODUCER','ENERGY SOURCE','GENERATION (Megawatthours)']].copy()
annual_US_generation_df.columns =  ['YEAR','STATE','TYPE OF PRODUCER','ENERGY SOURCE','GENERATION Mwh']
#state_group = annual_US_generation_df.groupby(['YEAR', 'STATE'])
#state_group.head(20)
#state_group

annual_US_generation_df.head(10)

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION Mwh
0,1990,AK,Total Electric Power Industry,Total,"5,599,506"
1,1990,AK,Total Electric Power Industry,Coal,"510,573"
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,"974,521"
3,1990,AK,Total Electric Power Industry,Natural Gas,"3,466,261"
4,1990,AK,Total Electric Power Industry,Petroleum,"497,116"
5,1990,AK,Total Electric Power Industry,Wind,0
6,1990,AK,Total Electric Power Industry,Wood and Wood Derived Fuels,"151,035"
7,1990,AK,"Electric Generators, Electric Utilities",Total,"4,493,024"
8,1990,AK,"Electric Generators, Electric Utilities",Coal,"311,960"
9,1990,AK,"Electric Generators, Electric Utilities",Hydroelectric Conventional,"974,521"


### CSV Source 2

In [ ]:
# read from local file
raw_csv_2 = "./Resources/name2.csv"
CSV_df_2 = pd.read_csv(raw_csv_1, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
CSV_df_2.head()

### CSV Source 3

In [ ]:
## Source 3
# read from local file
raw_csv_3 = "./Resources/name3.csv"
CSV_df_3 = pd.read_csv(raw_csv_1, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
CSV_df_3.head()

## Add CSV and json to SQL, SQLite

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table,Column,Integer,String
import glob
import os
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper

In [2]:
import csv, sqlite3

In [3]:
connection = sqlite3.connect("sqlite:///Energy_project.db") # change to 'sqlite:///your_filename.db' #:memory:
cursor = connection.cursor()



OperationalError: unable to open database file

In [18]:
# use your column names here
cursor.execute("CREATE TABLE t (State, Nuclear, Coal, Natural Gas, Petroleum, Hydro, Geothermal, Solar PV, Wind, Biomass Other") 

with open('./Resources/US/us_percentage.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i["State"], i["Nuclear"], i["Coal"], i["Natural Gas"], i["Petroleum"], i["Hydro"], i["Geothermal"], i["Solar - PV"], i["Wind"], i["Biomass and Other"]) for i in dr]

cur.executemany("INSERT INTO t (State, Nuclear, Coal, Natural Gas, Petroleum, Hydro, Geothermal, Solar - PV, Wind, Biomass and Other) ) VALUES (?, ?);", to_db)
con.commit()
con.close()

OperationalError: incomplete input

## Add CSV and json to SQL, Postgress

In [10]:
# Connect to local database
code_source_passcode = open('/Users/Richa/Desktop/Files/SQL_private_connect.py') 
# Add you password here
#code_source_passcode = open('/Users/......./SQL_private_connect.py') 

sql_private_connect = code_source_passcode.read()

In [11]:
engine = create_engine(f'postgresql://{sql_private_connect}@localhost:5432/Energy_DB')
connection = engine.connect()

In [12]:
# Check for tables
engine.table_names() 

['us_percentage', 'us_generation_power']

In [13]:
# Use pandas to load web scraped converted DataFrame into database
US_percentage_df.to_sql(name='us_percentage', 
                      con=engine, 
                      if_exists='replace', 
                      index=False)


In [14]:
# Use pandas to load csv converted DataFrame into database
annual_US_generation_df.to_sql(name='us_generation_power', 
                               con=engine, 
                               if_exists='replace', 
                               index=False)

In [ ]:
# # Use pandas to load csv converted DataFrame into database
# CSV_df_2.to_sql(name='name2', 
#                con=engine, 
#                if_exists='replace', 
#                index=False)

In [15]:
# Check for tables
engine.table_names() 

['us_percentage', 'us_generation_power']

### Check tables and data before making charts

In [16]:
# Confirm data has been added by querying the insider table
SQL_Web_scrapped_loaded = pd.read_sql_query('select * from us_percentage', con=engine)
SQL_Web_scrapped_loaded.head()

,State,Nuclear,Coal,Natural Gas,Petroleum,Hydro,Geothermal,Solar - PV,Wind,Biomass and Other
0,Alabama,31.4,19.2,38.7,0.0,7.8,0.0,0.3,0.0,2.5
1,Alaska,0.0,10.6,49.9,14.4,22.0,0.0,0.0,2.4,0.6
2,Arizona,28.1,20.4,40.9,0.1,5.4,0.0,4.5,0.5,0.2
3,Arkansas,22.0,37.9,33.0,0.1,4.5,0.0,0.3,0.0,2.2
4,California,7.9,0.1,42.4,0.0,19.5,5.6,14.0,7.3,3.2


In [17]:
# Confirm data has been added by querying the insider table
SQL_CSV1_loaded = pd.read_sql_query('select * from us_generation_power', con=engine)
SQL_CSV1_loaded.head()

,YEAR,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION Mwh
0,1990,AK,Total Electric Power Industry,Total,"5,599,506"
1,1990,AK,Total Electric Power Industry,Coal,"510,573"
2,1990,AK,Total Electric Power Industry,Hydroelectric Conventional,"974,521"
3,1990,AK,Total Electric Power Industry,Natural Gas,"3,466,261"
4,1990,AK,Total Electric Power Industry,Petroleum,"497,116"


In [ ]:
# # Confirm data has been added by querying the insider table
# SQL_CSV1_loaded = pd.read_sql_query('select * from name1', con=engine)
# SQL_CSV1_loaded.head()